In [1]:
import numpy as np
import scipy.stats as stats
import csv
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import gensim as gs 
from gensim import corpora, models, similarities
import logging
import multiprocessing

import pickle

import gensim

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, make_scorer, recall_score,precision_score,fbeta_score

np.random.seed(0)

# this line tells jupyter notebook to put the plots in the notebook rather than saving them to file.
%matplotlib inline

# this line makes plots prettier on mac retina screens. If you don't have one it shouldn't do anything.
%config InlineBackend.figure_format = 'retina'

cores = multiprocessing.cpu_count()

# from pomegranate import NaiveBayes, NormalDistribution
# import pomegranate

C:\Anaconda\envs\py36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Semi-Supervised Learning
<h2> Due to the fact that CPLELearner is written in Python 2.7, please be advised to run this notebook with Python 2 kernel </h2>

https://github.com/tmadl/semisup-learn <br>
pomegranate <br>
Pseudo-Labelling <br>
Sklearn Label Spread/Propagating

In [54]:
true_fake_vec = pd.read_csv('./csv/true_fake_vec.csv', index_col=False)

# unlabelled_full_vec = pd.read_csv('./csv/unlabelled_full_vec.csv')
all_y = pd.read_csv('./csv/target.csv', index_col=False)

X_train, X_test, y_train, y_test = train_test_split(true_fake_vec, all_y, train_size=0.75, random_state=8)

C:\Anaconda\envs\py36\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [57]:
new_index = ['s_'+ str(i) for i in true_fake_vec.index]
true_fake_vec['index2'] = new_index
cols = ['index2']
for a in true_fake_vec.columns[:-1]:
    cols.append(a)
# To rearrange the column names
true_fake_vec = true_fake_vec[cols]
true_fake_vec.to_csv('./csv/true_fake_vec_nohead.csv',index=False, header=False)

In [58]:
# true_fake_vec.drop(columns=['index2'],inplace=True)
true_fake_vec.drop(columns=['Unnamed: 0'],inplace=True)
# true_fake_vec.to_csv('./csv/true_fake_vec.csv')
true_fake_vec.head(3)

,index2,0,1,2,3,4,5,6,7,8,...,weapon,children,monster,ocean,giving,contentment,writing,rural,positive_emotion,musical
0,s_0,0.068758,0.240299,0.205623,0.177311,-0.139906,-0.092154,0.047262,-0.180129,0.023823,...,0.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0
1,s_1,-0.261129,0.018109,0.244408,0.246589,0.226827,0.183266,0.396658,0.242541,-0.140729,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,s_2,-0.033444,0.152059,-0.031646,0.260323,-0.046958,0.075744,0.150471,0.378979,0.078745,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0


In [3]:
unlabelled_index = list(range(unlabelled_full_vec.shape[0]))
np.random.shuffle(unlabelled_index)
kratio = 5
rand_unlabelled_vec = pd.DataFrame([unlabelled_full_vec.iloc[a] for a in unlabelled_index[:(len(true_fake_vec))*kratio]])

In [7]:
y_unlabelled = pd.DataFrame([-1] * rand_unlabelled_vec.shape[0], columns=['deceptive'])
X_train_new = pd.concat([X_train, rand_unlabelled_vec], axis=0).as_matrix()
y_train_new = pd.concat([y_train, y_unlabelled], axis = 0).T.as_matrix()[0]
# y_train_new = [a[0] for a in y_train_new]

In [4]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, make_scorer, fbeta_score

xg = XGBClassifier(colsample_bytree=0.25, max_depth=8, subsample=0.7, min_child_weight=3, gamma=1)
scorer = make_scorer(fbeta_score,beta=0.5)
scorelist = cross_val_score(xg, X_train, y_train, cv=5, scoring=scorer,n_jobs=-1)
print(scorelist, np.mean(scorelist))

C:\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(array([0.87521664, 0.88132635, 0.87603306, 0.87603306, 0.87993421]), 0.8777086633081931)


In [12]:
xg = XGBClassifier(colsample_bytree=0.25, max_depth=8, subsample=0.7, min_child_weight=3, gamma=1)
xg.fit(X_train, y_train)
best_pred = xg.predict(X_test)
print(classification_report(y_test, best_pred))

             precision    recall  f1-score   support

          0       0.86      0.87      0.86       204
          1       0.86      0.85      0.86       196

avg / total       0.86      0.86      0.86       400



C:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
from frameworks.CPLELearning import CPLELearningModel
from frameworks.SelfLearning import SelfLearningModel

In [ ]:
import sklearn
ssmodel2 = CPLELearningModel(sklearn.svm.SVC(kernel="rbf", probability=True), predict_from_probabilities=True) # RBF SVM
ssmodel2.fit(X_train_new, y_train_new)

n

In [ ]:
best_pred = ssmodel2.predict(X_test.as_matrix())
print(classification_report(y_test.as_matrix(), best_pred))

In [ ]:
ssmodel = CPLELearningModel (xg)
ssmodel.fit(X_train_new, y_train_new)
best_pred = ssmodel.predict(X_test.as_matrix())
print(classification_report(y_test.as_matrix(), best_pred))

In [ ]:
# Naive Bayes 
semi_model = pomegranate.NaiveBayes.from_samples(NormalDistribution, X_train_new, y_train_new, verbose=True,)

In [ ]:
best_pred = semi_model.predict(X_test)
print(classification_report(y_test, best_pred))

In [ ]:
from sklearn.semi_supervised import label_propagation
from scipy.sparse import csgraph

#Label Propagation
label_spread = label_propagation.LabelSpreading(kernel='rbf', alpha=0.1, gamma=0.2, n_jobs=-1)
label_spread.fit(X_train_new, y_train_new)
best_pred = label_spread.predict(X_test)
print(classification_report(y_test, best_pred))

In [ ]:
label_spread = label_propagation.LabelSpreading(kernel='knn', alpha=0.95, n_neighbors=40, n_jobs=-1)
label_spread.fit(X_train_new, y_train_new)
best_pred = label_spread.predict(X_test)
print(classification_report(y_test, best_pred))

In [ ]:
label_prop = label_propagation.LabelPropagation(kernel='rbf', gamma=0.2, n_jobs=-1)
label_prop.fit(X_train_new, y_train_new)
best_pred = label_prop.predict(X_test)
print(classification_report(y_test, best_pred))